# Feature Selection with Genetic Algorithm (GA)

The search space for this problem corresponds to the power set of the set of the 54 attributes, which means
$2^{54}$ possibilities. Denote such space by $\mathcal{S}$. A local search algorithm can seek the desired subset of attributes without having
to pass through each of the elements in this colossus space. The genetic algorithm treats solutions as
chromosomes, creating and operating in a fixed population. Operations are: selection, crossing-over
and mutation. Crossing-over allows the combination of individuals with some probability (commonly set to 1), hoping that 
good features will be propagated. Mutation allows variation in the population and is represented
by a small probability, that may vary during the execution, depending on the implementation.

Before submitting this problem to the GA algorithm, it is necessary to define the way solutions (subsets) will
be represented and the function to rank the solutions, also known as the fitness function.

## Solutions representation

For this problem, the chromosome representation is a vector $x^i \in \{0,1\}^{54}$, in which each component $j$,  $x^i_j$, means the presence of the attribute $j$ in the solution $i$.

## Fitness function

This function will rank each solution according to the objective of selecting the most appropriate attribute for this classification problem. Since the purpose is to determine the class of an instance based on the attribute values, the similarity measure of correlation will be applied here. Correlation between two random variables $X$ and $Y$ is given by the
equation:

$$
corr(X,Y) = \frac{cov(X,Y)}{\sigma_X\sigma_Y}
$$

where $cov(X,Y)$ is the covariance between $X$ and $Y$, and $\sigma_X, \sigma_Y$ are the stardard deviations for $X$ and $Y$ respectively. The fitness function is defined as the mean of the unsigned correlations (in $[0,1]$) of each selected attribute with respect to the class:

$$
f(x^i) = \frac{\sum_{j, x^i_j = 1} |corr(x^i_j,cover\_type)|}{\sum_{j, x^i_j=1} 1}.
$$

## Implementation

First of all, it is necessary to import libraries and load the dataset:

In [1]:
import pandas as pd
import numpy as np
from pyeasyga.pyeasyga import GeneticAlgorithm

# read the dataset
dataset = pd.read_csv("datasets/covertype_norm_train.csv")
# preview of dataset
dataset.head()

,elevation,aspect,slope,horiz_dist_hydro,vert_dist_hydro,horiz_dist_road,hillshade_9,hill_shade_noon,hill_shade_15,horiz_dist_fire,...,soil_type_31,soil_type_32,soil_type_33,soil_type_34,soil_type_35,soil_type_36,soil_type_37,soil_type_38,soil_type_39,cover_type
0,-0.573753,-0.518424,-0.428658,0.436024,-0.475092,-0.979056,0.927864,0.144520,-0.534162,-0.220768,...,-0.214265,-0.202489,-0.039088,-0.081433,-0.016657,-0.044107,-0.220216,-0.219696,-0.172986,3
1,1.656009,-0.010549,0.868502,-0.516497,-0.280544,1.817610,0.862413,0.665801,-0.534162,2.273548,...,-0.214265,4.938531,-0.039088,-0.081433,-0.016657,-0.044107,-0.220216,-0.219696,-0.172986,7
2,0.169501,-0.799569,0.632655,0.455170,1.891910,-0.388051,0.796962,-1.245563,-1.335438,-0.687429,...,-0.214265,-0.202489,-0.039088,-0.081433,-0.016657,-0.044107,-0.220216,-0.219696,-0.172986,5
3,-1.205043,1.268208,1.576043,0.234990,1.648725,-0.649457,-2.933743,-0.159560,1.956291,-0.501856,...,-0.214265,-0.202489,-0.039088,-0.081433,-0.016657,-0.044107,-0.220216,-0.219696,-0.172986,6
4,-1.057345,0.152697,0.986425,0.134472,0.530073,-1.041945,0.404256,1.056762,-0.014415,-0.794770,...,-0.214265,-0.202489,-0.039088,-0.081433,-0.016657,-0.044107,-0.220216,-0.219696,-0.172986,3


Correlations with respect to the class are fixed values, and can be calculated using the following:

In [2]:
# class correlations
class_correlations = dataset.corr(method="pearson")['cover_type']
print(class_correlations)

# Replacing NaN values with 0
class_correlations.fillna(0, inplace=True)

elevation           0.017763
aspect             -0.003926
slope               0.091813
horiz_dist_hydro   -0.003953
vert_dist_hydro     0.065777
horiz_dist_road    -0.101594
hillshade_9        -0.013562
hill_shade_noon    -0.105797
hill_shade_15      -0.056238
horiz_dist_fire    -0.088208
wild_area_0        -0.223716
wild_area_1         0.000400
wild_area_2         0.123695
wild_area_3         0.074808
soil_type_0         0.009627
soil_type_1         0.011772
soil_type_2        -0.020670
soil_type_3        -0.027416
soil_type_4         0.028314
soil_type_5         0.014984
soil_type_6              NaN
soil_type_7              NaN
soil_type_8        -0.030136
soil_type_9         0.133024
soil_type_10        0.008588
soil_type_11       -0.126998
soil_type_12        0.043901
soil_type_13        0.020771
soil_type_14             NaN
soil_type_15        0.012799
soil_type_16        0.038908
soil_type_17       -0.005356
soil_type_18       -0.048150
soil_type_19       -0.034100
soil_type_20  

Now, we define the fitness function:

In [3]:
data = class_correlations[:54].tolist()

# Fitness function
def fitness (individual, data):
    '''
    Compute the fitness of a solution.
    '''
    if individual.count(1) > 0:
        mean = data[0]
        for i in range(len(individual)):
            if individual[i] == 1:
                mean += abs(data[i])
        return (mean/individual.count(1))
    return 0.0

# Dataframe to store results
columns = ['population', 'max_generations', \
           'combination_prob', 'mutation_prob', 'iteration']
columns = columns + class_correlations[:54].index.tolist()

results = pd.DataFrame(columns=columns)

Now, we run the algorithm, using different parameter values:

In [4]:
populations  = [25, 50, 100]
generations  = [50,100,200]
combinations = [.7, .8, .9]
mutations    = [.05, .1, .15]
iterations   = 30

# Algorithm definition 
ga = GeneticAlgorithm(data, elitism=True, maximise_fitness=True)

# Use own fitness function
ga.fitness_function = fitness

index = 0

for population in populations:
    ga.population_size = population
    for generation in generations:
        ga.generations = generation
        for combination in combinations:
            ga.crossover_probability = combination
            for mutation in mutations:
                ga.mutation_probability = mutation
                for i in range(iterations):
                    # Running genetic algorithm
                    ga.run()
                    print("pop: ", population, "generation: ", generation, \
                          "comb: ", combination, "mu: ", mutation, "it: ", (i+1))
                    r = [population, generation, combination, mutation, i+1] +\
                    [x==1 for x in ga.best_individual()[1]]
                    results.loc[index] = r
                    index += 1
                    

pop:  25 generation:  50 comb:  0.7 mu:  0.05 it:  1
pop:  25 generation:  50 comb:  0.7 mu:  0.05 it:  2
pop:  25 generation:  50 comb:  0.7 mu:  0.05 it:  3
pop:  25 generation:  50 comb:  0.7 mu:  0.05 it:  4
pop:  25 generation:  50 comb:  0.7 mu:  0.05 it:  5
pop:  25 generation:  50 comb:  0.7 mu:  0.05 it:  6
pop:  25 generation:  50 comb:  0.7 mu:  0.05 it:  7
pop:  25 generation:  50 comb:  0.7 mu:  0.05 it:  8
pop:  25 generation:  50 comb:  0.7 mu:  0.05 it:  9
pop:  25 generation:  50 comb:  0.7 mu:  0.05 it:  10
pop:  25 generation:  50 comb:  0.7 mu:  0.05 it:  11
pop:  25 generation:  50 comb:  0.7 mu:  0.05 it:  12
pop:  25 generation:  50 comb:  0.7 mu:  0.05 it:  13
pop:  25 generation:  50 comb:  0.7 mu:  0.05 it:  14
pop:  25 generation:  50 comb:  0.7 mu:  0.05 it:  15
pop:  25 generation:  50 comb:  0.7 mu:  0.05 it:  16
pop:  25 generation:  50 comb:  0.7 mu:  0.05 it:  17
pop:  25 generation:  50 comb:  0.7 mu:  0.05 it:  18
pop:  25 generation:  50 comb:  0.7 m

pop:  25 generation:  50 comb:  0.8 mu:  0.15 it:  5
pop:  25 generation:  50 comb:  0.8 mu:  0.15 it:  6
pop:  25 generation:  50 comb:  0.8 mu:  0.15 it:  7
pop:  25 generation:  50 comb:  0.8 mu:  0.15 it:  8
pop:  25 generation:  50 comb:  0.8 mu:  0.15 it:  9
pop:  25 generation:  50 comb:  0.8 mu:  0.15 it:  10
pop:  25 generation:  50 comb:  0.8 mu:  0.15 it:  11
pop:  25 generation:  50 comb:  0.8 mu:  0.15 it:  12
pop:  25 generation:  50 comb:  0.8 mu:  0.15 it:  13
pop:  25 generation:  50 comb:  0.8 mu:  0.15 it:  14
pop:  25 generation:  50 comb:  0.8 mu:  0.15 it:  15
pop:  25 generation:  50 comb:  0.8 mu:  0.15 it:  16
pop:  25 generation:  50 comb:  0.8 mu:  0.15 it:  17
pop:  25 generation:  50 comb:  0.8 mu:  0.15 it:  18
pop:  25 generation:  50 comb:  0.8 mu:  0.15 it:  19
pop:  25 generation:  50 comb:  0.8 mu:  0.15 it:  20
pop:  25 generation:  50 comb:  0.8 mu:  0.15 it:  21
pop:  25 generation:  50 comb:  0.8 mu:  0.15 it:  22
pop:  25 generation:  50 comb:  0

pop:  25 generation:  100 comb:  0.7 mu:  0.1 it:  8
pop:  25 generation:  100 comb:  0.7 mu:  0.1 it:  9
pop:  25 generation:  100 comb:  0.7 mu:  0.1 it:  10
pop:  25 generation:  100 comb:  0.7 mu:  0.1 it:  11
pop:  25 generation:  100 comb:  0.7 mu:  0.1 it:  12
pop:  25 generation:  100 comb:  0.7 mu:  0.1 it:  13
pop:  25 generation:  100 comb:  0.7 mu:  0.1 it:  14
pop:  25 generation:  100 comb:  0.7 mu:  0.1 it:  15
pop:  25 generation:  100 comb:  0.7 mu:  0.1 it:  16
pop:  25 generation:  100 comb:  0.7 mu:  0.1 it:  17
pop:  25 generation:  100 comb:  0.7 mu:  0.1 it:  18
pop:  25 generation:  100 comb:  0.7 mu:  0.1 it:  19
pop:  25 generation:  100 comb:  0.7 mu:  0.1 it:  20
pop:  25 generation:  100 comb:  0.7 mu:  0.1 it:  21
pop:  25 generation:  100 comb:  0.7 mu:  0.1 it:  22
pop:  25 generation:  100 comb:  0.7 mu:  0.1 it:  23
pop:  25 generation:  100 comb:  0.7 mu:  0.1 it:  24
pop:  25 generation:  100 comb:  0.7 mu:  0.1 it:  25
pop:  25 generation:  100 comb

pop:  25 generation:  100 comb:  0.9 mu:  0.05 it:  9
pop:  25 generation:  100 comb:  0.9 mu:  0.05 it:  10
pop:  25 generation:  100 comb:  0.9 mu:  0.05 it:  11
pop:  25 generation:  100 comb:  0.9 mu:  0.05 it:  12
pop:  25 generation:  100 comb:  0.9 mu:  0.05 it:  13
pop:  25 generation:  100 comb:  0.9 mu:  0.05 it:  14
pop:  25 generation:  100 comb:  0.9 mu:  0.05 it:  15
pop:  25 generation:  100 comb:  0.9 mu:  0.05 it:  16
pop:  25 generation:  100 comb:  0.9 mu:  0.05 it:  17
pop:  25 generation:  100 comb:  0.9 mu:  0.05 it:  18
pop:  25 generation:  100 comb:  0.9 mu:  0.05 it:  19
pop:  25 generation:  100 comb:  0.9 mu:  0.05 it:  20
pop:  25 generation:  100 comb:  0.9 mu:  0.05 it:  21
pop:  25 generation:  100 comb:  0.9 mu:  0.05 it:  22
pop:  25 generation:  100 comb:  0.9 mu:  0.05 it:  23
pop:  25 generation:  100 comb:  0.9 mu:  0.05 it:  24
pop:  25 generation:  100 comb:  0.9 mu:  0.05 it:  25
pop:  25 generation:  100 comb:  0.9 mu:  0.05 it:  26
pop:  25 ge

pop:  25 generation:  200 comb:  0.7 mu:  0.15 it:  10
pop:  25 generation:  200 comb:  0.7 mu:  0.15 it:  11
pop:  25 generation:  200 comb:  0.7 mu:  0.15 it:  12
pop:  25 generation:  200 comb:  0.7 mu:  0.15 it:  13
pop:  25 generation:  200 comb:  0.7 mu:  0.15 it:  14
pop:  25 generation:  200 comb:  0.7 mu:  0.15 it:  15
pop:  25 generation:  200 comb:  0.7 mu:  0.15 it:  16
pop:  25 generation:  200 comb:  0.7 mu:  0.15 it:  17
pop:  25 generation:  200 comb:  0.7 mu:  0.15 it:  18
pop:  25 generation:  200 comb:  0.7 mu:  0.15 it:  19
pop:  25 generation:  200 comb:  0.7 mu:  0.15 it:  20
pop:  25 generation:  200 comb:  0.7 mu:  0.15 it:  21
pop:  25 generation:  200 comb:  0.7 mu:  0.15 it:  22
pop:  25 generation:  200 comb:  0.7 mu:  0.15 it:  23
pop:  25 generation:  200 comb:  0.7 mu:  0.15 it:  24
pop:  25 generation:  200 comb:  0.7 mu:  0.15 it:  25
pop:  25 generation:  200 comb:  0.7 mu:  0.15 it:  26
pop:  25 generation:  200 comb:  0.7 mu:  0.15 it:  27
pop:  25 g

...

pop:  100 generation:  50 comb:  0.9 mu:  0.05 it:  23
pop:  100 generation:  50 comb:  0.9 mu:  0.05 it:  24
pop:  100 generation:  50 comb:  0.9 mu:  0.05 it:  25
pop:  100 generation:  50 comb:  0.9 mu:  0.05 it:  26
pop:  100 generation:  50 comb:  0.9 mu:  0.05 it:  27
pop:  100 generation:  50 comb:  0.9 mu:  0.05 it:  28
pop:  100 generation:  50 comb:  0.9 mu:  0.05 it:  29
pop:  100 generation:  50 comb:  0.9 mu:  0.05 it:  30
pop:  100 generation:  50 comb:  0.9 mu:  0.1 it:  1
pop:  100 generation:  50 comb:  0.9 mu:  0.1 it:  2
pop:  100 generation:  50 comb:  0.9 mu:  0.1 it:  3
pop:  100 generation:  50 comb:  0.9 mu:  0.1 it:  4
pop:  100 generation:  50 comb:  0.9 mu:  0.1 it:  5
pop:  100 generation:  50 comb:  0.9 mu:  0.1 it:  6
pop:  100 generation:  50 comb:  0.9 mu:  0.1 it:  7
pop:  100 generation:  50 comb:  0.9 mu:  0.1 it:  8
pop:  100 generation:  50 comb:  0.9 mu:  0.1 it:  9
pop:  100 generation:  50 comb:  0.9 mu:  0.1 it:  10
pop:  100 generation:  50 com

pop:  100 generation:  100 comb:  0.7 mu:  0.15 it:  23
pop:  100 generation:  100 comb:  0.7 mu:  0.15 it:  24
pop:  100 generation:  100 comb:  0.7 mu:  0.15 it:  25
pop:  100 generation:  100 comb:  0.7 mu:  0.15 it:  26
pop:  100 generation:  100 comb:  0.7 mu:  0.15 it:  27
pop:  100 generation:  100 comb:  0.7 mu:  0.15 it:  28
pop:  100 generation:  100 comb:  0.7 mu:  0.15 it:  29
pop:  100 generation:  100 comb:  0.7 mu:  0.15 it:  30
pop:  100 generation:  100 comb:  0.8 mu:  0.05 it:  1
pop:  100 generation:  100 comb:  0.8 mu:  0.05 it:  2
pop:  100 generation:  100 comb:  0.8 mu:  0.05 it:  3
pop:  100 generation:  100 comb:  0.8 mu:  0.05 it:  4
pop:  100 generation:  100 comb:  0.8 mu:  0.05 it:  5
pop:  100 generation:  100 comb:  0.8 mu:  0.05 it:  6
pop:  100 generation:  100 comb:  0.8 mu:  0.05 it:  7
pop:  100 generation:  100 comb:  0.8 mu:  0.05 it:  8
pop:  100 generation:  100 comb:  0.8 mu:  0.05 it:  9
pop:  100 generation:  100 comb:  0.8 mu:  0.05 it:  10
p

pop:  100 generation:  100 comb:  0.9 mu:  0.1 it:  21
pop:  100 generation:  100 comb:  0.9 mu:  0.1 it:  22
pop:  100 generation:  100 comb:  0.9 mu:  0.1 it:  23
pop:  100 generation:  100 comb:  0.9 mu:  0.1 it:  24
pop:  100 generation:  100 comb:  0.9 mu:  0.1 it:  25
pop:  100 generation:  100 comb:  0.9 mu:  0.1 it:  26
pop:  100 generation:  100 comb:  0.9 mu:  0.1 it:  27
pop:  100 generation:  100 comb:  0.9 mu:  0.1 it:  28
pop:  100 generation:  100 comb:  0.9 mu:  0.1 it:  29
pop:  100 generation:  100 comb:  0.9 mu:  0.1 it:  30
pop:  100 generation:  100 comb:  0.9 mu:  0.15 it:  1
pop:  100 generation:  100 comb:  0.9 mu:  0.15 it:  2
pop:  100 generation:  100 comb:  0.9 mu:  0.15 it:  3
pop:  100 generation:  100 comb:  0.9 mu:  0.15 it:  4
pop:  100 generation:  100 comb:  0.9 mu:  0.15 it:  5
pop:  100 generation:  100 comb:  0.9 mu:  0.15 it:  6
pop:  100 generation:  100 comb:  0.9 mu:  0.15 it:  7
pop:  100 generation:  100 comb:  0.9 mu:  0.15 it:  8
pop:  100 

pop:  100 generation:  200 comb:  0.8 mu:  0.05 it:  19
pop:  100 generation:  200 comb:  0.8 mu:  0.05 it:  20
pop:  100 generation:  200 comb:  0.8 mu:  0.05 it:  21
pop:  100 generation:  200 comb:  0.8 mu:  0.05 it:  22
pop:  100 generation:  200 comb:  0.8 mu:  0.05 it:  23
pop:  100 generation:  200 comb:  0.8 mu:  0.05 it:  24
pop:  100 generation:  200 comb:  0.8 mu:  0.05 it:  25
pop:  100 generation:  200 comb:  0.8 mu:  0.05 it:  26
pop:  100 generation:  200 comb:  0.8 mu:  0.05 it:  27
pop:  100 generation:  200 comb:  0.8 mu:  0.05 it:  28
pop:  100 generation:  200 comb:  0.8 mu:  0.05 it:  29
pop:  100 generation:  200 comb:  0.8 mu:  0.05 it:  30
pop:  100 generation:  200 comb:  0.8 mu:  0.1 it:  1
pop:  100 generation:  200 comb:  0.8 mu:  0.1 it:  2
pop:  100 generation:  200 comb:  0.8 mu:  0.1 it:  3
pop:  100 generation:  200 comb:  0.8 mu:  0.1 it:  4
pop:  100 generation:  200 comb:  0.8 mu:  0.1 it:  5
pop:  100 generation:  200 comb:  0.8 mu:  0.1 it:  6
pop:

pop:  100 generation:  200 comb:  0.9 mu:  0.15 it:  18
pop:  100 generation:  200 comb:  0.9 mu:  0.15 it:  19
pop:  100 generation:  200 comb:  0.9 mu:  0.15 it:  20
pop:  100 generation:  200 comb:  0.9 mu:  0.15 it:  21
pop:  100 generation:  200 comb:  0.9 mu:  0.15 it:  22
pop:  100 generation:  200 comb:  0.9 mu:  0.15 it:  23
pop:  100 generation:  200 comb:  0.9 mu:  0.15 it:  24
pop:  100 generation:  200 comb:  0.9 mu:  0.15 it:  25
pop:  100 generation:  200 comb:  0.9 mu:  0.15 it:  26
pop:  100 generation:  200 comb:  0.9 mu:  0.15 it:  27
pop:  100 generation:  200 comb:  0.9 mu:  0.15 it:  28
pop:  100 generation:  200 comb:  0.9 mu:  0.15 it:  29
pop:  100 generation:  200 comb:  0.9 mu:  0.15 it:  30


Finally, we store the results in a CSV:

In [5]:
# Create CSV with new dataset
results.to_csv('results/ga_selected_attributes.csv', index=False)